In [1]:
import numpy
import math
from scipy import stats
from scipy.optimize import minimize
%matplotlib inline

Плотности данные в задаче:

$$f_0(x, a, b) = \frac{3b^3}{(x-a)^4}I\{x>a+b\}$$

$$f_1(x, a, b) = \frac{1}{2b}\exp(-\frac{|x-a|}{b})$$

Все, что связано с вероятностями в задаче будем считать путем генерации большого количества выборок из одного распределения и усреднения значений по ним.

In [2]:
# Функции для генериции выборок в наших обозначениях

def pareto_sample(a, b, N):
    return stats.pareto.rvs(b=3, loc=a, scale=b, size=N)

def laplace_sample(a, b, N):
    return stats.laplace.rvs(loc=a, scale=b, size=N)

In [3]:
# Правдоподобие для Парето и Лапласа. Не используется, нужно только для сравнения с функциями из следующего блока.

def pareto(sample, a, b):
    if sample.min() <= a + b:
        return 0
    else:
        return ((3 * b ** 3) ** len(sample)) / numpy.prod(numpy.array([math.pow((x - a), 4) for x in sample]))

def laplace(sample, a, b):
    return numpy.exp(-numpy.sum(numpy.array([numpy.absolute(x - a) for x in sample])) / b) / (2 * b) ** len(sample)

In [4]:
# Корни n-ой степени правдоподобий. Используем их, чтобы числа не получались слишком большими

def nroot_pareto(sample, a, b):
    if sample.min() <= a + b:
        return 0
    else:
        return (3 * b ** 3) / numpy.prod(numpy.array([math.pow((x - a), 4 / len(sample)) for x in sample]))

def nroot_laplace(sample, a, b):
    return numpy.exp(-numpy.sum(numpy.array([numpy.absolute(x - a) for x in sample])) / (b * len(sample))) / (2 * b)

In [5]:
# Корень n-ой степени из RML. Считаем максимумы спомощью scipy.optimize
# (тут еще нужен подгон начальных приближений, чтобы оптимизация нормально сработала)

def nroot_RML(sample):
    laplace_optimization = minimize(
        lambda params, sample: -nroot_laplace(sample, params[0], params[1]),
        [numpy.min(sample) - 10, 1], args=sample, bounds=((None, None), (0, None)), method='Powell'
    )
    pareto_optimization = minimize(
        lambda params, sample: -nroot_pareto(sample, params[0], params[1]),
        [numpy.min(sample) - 10, 1], args=sample, bounds=((None, None), (0, None)), method='Powell'
    )
    
    return laplace_optimization['fun'] / pareto_optimization['fun']

In [6]:
# Частота выполнения критерия для заданных выборок и заданной квантили. Нужно для рассчета мощности критерия.

def probability(samples, u):
    return numpy.mean([(1 if (nroot_RML(sample) >  u) else 0) for sample in samples])

In [7]:
# Поскольку по условию RML не зависит от a и b, возмем их какими-нибудь, например 1, 1

alphas = [0.1, 0.05, 0.01]
ns = [50, 100, 200]
bootstreps_length = 1000
a = 1
b = 1

# Генерируем много выборок из распределения с плотностью f_0 чтобы посчитать квантили.
pareto_bootstreps = [
    [pareto_sample(a, b, n) for i in range(bootstreps_length)]
    for n in ns
]

In [8]:
quantilies = []

# Для каждого alpha и n оцениваем квантили с помощью сортировки массива RML для разных выборок.
for pareto_bootstrep in pareto_bootstreps:
    for alpha in alphas:
        RMLs = [nroot_RML(sample) for sample in pareto_bootstrep]
        RMLs.sort()
        
        # Для каждой полученной квантили генерируем N выборок из распределения с плотностью f_1,
        # чтобы оценить мощность критерия
        quantilies.append((
                len(pareto_bootstrep[0]), RMLs[int((1 - alpha) * len(RMLs))],
                [laplace_sample(a, b, len(pareto_bootstrep[0])) for i in range(bootstreps_length)]
            )
        )
        print("alpha={}, n={}, u={}".format(alpha, len(pareto_bootstrep[0]), RMLs[int((1 - alpha) * len(RMLs))]))

/usr/lib/python3.5/site-packages/scipy/optimize/_minimize.py:394: RuntimeWarning: Method Powell cannot handle constraints nor bounds.
  RuntimeWarning)
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: overflow encountered in exp


alpha=0.1, n=50, u=0.7199872659866569
alpha=0.05, n=50, u=0.7576283978334849
alpha=0.01, n=50, u=0.810809642389608
alpha=0.1, n=100, u=0.6824746039325952
alpha=0.05, n=100, u=0.7122449775052603
alpha=0.01, n=100, u=0.7548716857819472
alpha=0.1, n=200, u=0.662685178589413
alpha=0.05, n=200, u=0.6775784572854451
alpha=0.01, n=200, u=0.7032138088979488


In [9]:
probabilities = []

# Считаем мощности критериев для всех квантилей
for quantile in quantilies:
    capacity = probability(quantile[2], quantile[1])
    probabilities.append(capacity)
    
    print("n={}, u={}, p={}".format(quantile[0], quantile[1], capacity))

/usr/lib/python3.5/site-packages/scipy/optimize/_minimize.py:394: RuntimeWarning: Method Powell cannot handle constraints nor bounds.
  RuntimeWarning)


n=50, u=0.7199872659866569, p=1.0
n=50, u=0.7576283978334849, p=1.0
n=50, u=0.810809642389608, p=1.0
n=100, u=0.6824746039325952, p=1.0
n=100, u=0.7122449775052603, p=1.0
n=100, u=0.7548716857819472, p=1.0
n=200, u=0.662685178589413, p=1.0
n=200, u=0.6775784572854451, p=1.0
n=200, u=0.7032138088979488, p=1.0


Получили релультат, что мощность всегда равна единице. Это произошло из-за того, что значения корня из RML для Паретто всегда порядка $0.5$ а для Лапласа порядка $150$. Это означет, что если критерий выполнился для распределения Парето, то он с огромной вероятностью выполнится и для Лапласа. Получается, что такой критерий Легко отличает выборку Парето от выборки Лапласа. С таким различием в значениях RML противный результат был бы странным.

In [10]:
# Для рсравнения значений RML

print(nroot_RML(laplace_sample(a, b, 1000)), nroot_RML(pareto_sample(a, b, 1000)))

/usr/lib/python3.5/site-packages/scipy/optimize/_minimize.py:394: RuntimeWarning: Method Powell cannot handle constraints nor bounds.
  RuntimeWarning)


219.656803587 0.608241393602


Подтверждение предыдущих слов в блоке выше. Для другой пары распределений могло бы быть по-другому.